<a href="https://colab.research.google.com/github/pandov/cups-mail/blob/master/segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%capture
! sh colab.sh

In [ ]:
%%capture
from src.nn import set_grad_enabled, Runner, get_segmentation_components, get_loaders, score_aux
logdir = './logs/segmentation'
! rm -r {logdir}

In [ ]:
%load_ext tensorboard
%tensorboard --logdir $logdir

In [ ]:
class MultiRunner(Runner):
    def _handle_batch(self, batch):
        x, y = batch
        y_pred = self.model(x)
        is_train = self.is_train_loader == True

        with set_grad_enabled(is_train):
            loss = self.criterion(y_pred, y)
            aux = score_aux(y_pred, y)
            self.batch_metrics.update({
                'aux': aux,
                'loss': loss,
                'lr': self.state.scheduler.get_last_lr()[0],
            })

            if is_train:
                loss.backward()
                self.optimizer.step()
                self.optimizer.zero_grad()
            else:
                if self.loader_batch_step == self.loader_len:
                    self.scheduler.step()

In [ ]:
net, encoder = 'unet', 'timm-efficientnet-b4'
components = get_segmentation_components(net, encoder, 'adam', 'steplr')
components['loaders'] = get_loaders(batch_size=8, keys=['image', 'mask'])
runner = MultiRunner()
runner.train(
    logdir=f'{logdir}/{net}_{encoder}',
    num_epochs=120,
    # verbose=True,
    **components
)

In [ ]:
logzip = logdir + '.zip'
! zip -r {logzip} {logdir}
! mv {logzip} 'drive/My Drive/'
# ! git pull origin master
# ! git add logs
# ! git commit -m 'Changed from Colab'
# ! git push -u origin master